In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import torch
import torch.nn as nn 

from model.SpeechLP import SLP

from utils.Config import Config
from utils.MLS import MLSDataset
from utils.Trainer import Trainer
from utils.Processing import Processing

from torch.utils.data import DataLoader

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [9]:
Config.display()

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Text Settings:
  Max Token : 128 tokens

Model Settings:
  Model Name: DiTTO-TTS
  Embedding Dim: 1472
  Num Layers: 1
  Attention Heads: 1

Training Settings:
  Batch Size: 4
  Learning Rate: 0.0001
  Betas: [0.9, 0.999]
  Epochs: 10
  Nb samples: 10000
  Nb classes: 11
  Device: cuda

Data Settings:
  Train path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/train
  Test path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/test
  Dev path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/dev


In [10]:
# Processing.remove_metadata_from_audio_folder(Config.TRAIN_PATH+"/"+"audio", Config.TRAIN_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(Config.TEST_PATH+"/"+"audio", Config.TEST_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(Config.DEV_PATH+"/"+"audio", Config.DEV_PATH+"/"+"audio_clean",)

In [11]:
train_set = MLSDataset(
    data_dir=Config.TRAIN_PATH,
    max_text_token_length=Config.MAX_TOKEN_LENGTH,
    sampling_rate=Config.SAMPLE_RATE,
)

val_set = MLSDataset(
    data_dir=Config.DEV_PATH,
    max_text_token_length=Config.MAX_TOKEN_LENGTH,
    sampling_rate=Config.SAMPLE_RATE,
)


test_set = MLSDataset(
    data_dir=Config.TEST_PATH,
    max_text_token_length=Config.MAX_TOKEN_LENGTH,
    sampling_rate=Config.SAMPLE_RATE,
)

train_loader = DataLoader(train_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)
val_loader = DataLoader(val_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)
test_loader = DataLoader(test_set, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=MLSDataset.collate_fn)

Loading tokenized transcripts...
Loading tokenized transcripts...
Loading tokenized transcripts...


In [12]:
model = SLP(Config.NB_CLASSES, Config.NHEAD ,Config.NUM_LAYERS)
model = model.to(Config.DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW

trainer = Trainer()
trainer.set_model(model, name="SLP")\
    .set_criterion(criterion)\
    .set_optimizer(optimizer)\
    .fit(
        train_data=test_loader, validation_data=val_loader, 
        epochs=Config.EPOCHS, learning_rate=Config.LEARNING_RATE, checkpoint_interval=1        
    )

[INFO] Model's device is : cuda


  0%|          | 1/607 [00:00<07:33,  1.34it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 9.67 GiB of which 35.75 MiB is free. Including non-PyTorch memory, this process has 9.57 GiB memory in use. Of the allocated memory 8.91 GiB is allocated by PyTorch, and 409.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 